In [1]:
#%pip install sklearn
#%pip install torch

from math import sqrt
import os
from time import time
import random
import numpy as np
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim

## project structure
DATA_DIR = "/data/projects/capturingBias/research/framing/data/"  # change to "./" for current directory
DATA_NPZ = DATA_DIR + "data.npz"

## load files
data = np.load(DATA_NPZ)

X_2D = data['X_2D']
X_3D = data['X_3D']
y_likert_crowd = data['y_likert_crowd']
y_likert_experts = data['y_likert_experts']
y_dominant_crowd = data['y_dominant_crowd']
y_dominant_experts = data['y_dominant_experts']


# likert
likert_expert_idx = np.where(y_likert_experts > -1)[0]
likert_crowd_idx = np.setdiff1d(np.where(y_likert_crowd > -1)[0],
                                likert_expert_idx,
                                assume_unique=True)

# dominant
dominant_expert_idx = np.where(y_dominant_experts > -1)[0]
dominant_crowd_idx = np.setdiff1d(np.where(y_dominant_crowd > -1)[0],
                                  dominant_expert_idx,
                                  assume_unique=True)

In [2]:
def set_seed(seed=-1):
    if seed < 0:
        seed = np.random.randint(0, 2**32-1)

    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seed(47)  # make reproducable

In [3]:
def create_splits(y):
    train_idx = list()
    test_idx = list()
    
    strats = [np.where(y == lab)[0] for lab in np.unique(y) if lab > -1]
    for strat in strats:
        n = strat.shape[0]
        train_idx.append(strat[:int(n*0.8)])
        test_idx.append(strat[int(n*0.8):])
        
    train_idx = np.concatenate(train_idx)
    test_idx = np.concatenate(test_idx)
    
    np.random.shuffle(train_idx)
    np.random.shuffle(test_idx)
    
    return (train_idx, test_idx)

def create_splits_one_hot(y):
    vec = -np.ones(y.shape[0])
    nonzero = y.nonzero()
    vec[nonzero[:,0]] = nonzero[:,1].float()
    
    return create_splits(vec)

# Majority Class

In [4]:
from collections import Counter


def majority_class(y):
    ct = Counter(y)
    return ct.most_common(1)[0][1] / len(y)

In [5]:
majority_class_acc_crowd_likert = majority_class(y_likert_crowd[likert_crowd_idx])
majority_class_acc_experts_likert = majority_class(y_likert_experts[likert_expert_idx])

print("Majority class accuracy on Likert labels (baseline)")
print(" crowd labels:  {:.4f}".format(majority_class_acc_crowd_likert))
print(" expert labels: {:.4f}".format(majority_class_acc_experts_likert))

majority_class_acc_crowd_dominant = majority_class(y_dominant_crowd[dominant_crowd_idx])
majority_class_acc_experts_dominant = majority_class(y_dominant_experts[dominant_expert_idx])

print("\nMajority class accuracy on Dominant labels (baseline)")
print(" crowd labels:  {:.4f}".format(majority_class_acc_crowd_dominant))
print(" expert labels: {:.4f}".format(majority_class_acc_experts_dominant))

Majority class accuracy on Likert labels (baseline)
 crowd labels:  0.2903
 expert labels: 0.2414

Majority class accuracy on Dominant labels (baseline)
 crowd labels:  0.6000
 expert labels: 0.6818


# Random Forest (supervised)

We start with a traditional, or 'shallow', machine learning model: random forest. Because random forest does not support iterative learning, we test both the crowd and expert sets separately.

We use stratified cross validation to reduce the effects caused by the small size of the data set.

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


N_ESTIMATORS = [100, 250, 500, 750, 1000, 2000]
N_FOLDS = 10

def random_forest(X, y, index, n_folds=N_FOLDS, n_estimators=N_ESTIMATORS):
    n_samples = X[index].shape[0]
    for n_estimators in N_ESTIMATORS:
        print("Training with {} estimators".format(n_estimators))
        acc = 0
        for fold_i in range(N_FOLDS):
            print(" Starting fold {} / {}".format(fold_i+1, N_FOLDS), end='')
            
            train_fold_idx, test_fold_idx  = create_splits(y[index])
            train_idx = index[train_fold_idx]
            test_idx = index[test_fold_idx]
        
            model = RandomForestClassifier(n_estimators=n_estimators)
            model.fit(X[train_idx], y[train_idx])
            
            y_pred = model.predict(X[test_idx])
            fold_acc = accuracy_score(y[test_idx], y_pred)
        
            acc += fold_acc
            print(" (acc: {:.4f})".format(fold_acc))
            
        acc /= N_FOLDS
        print("Mean accuracy on test set: {:.4f}\n".format(acc))
    
    return acc

In [7]:
print("=== Results of supervised learning on expert likert labels ===")
random_forest_acc_experts_likert = random_forest(X_2D,
                                                 y_likert_experts, 
                                                 likert_expert_idx)

=== Results of supervised learning on expert likert labels ===
Training with 100 estimators
 Starting fold 1 / 10 (acc: 0.1333)
 Starting fold 2 / 10 (acc: 0.2667)
 Starting fold 3 / 10 (acc: 0.2000)
 Starting fold 4 / 10 (acc: 0.1333)
 Starting fold 5 / 10 (acc: 0.2667)
 Starting fold 6 / 10 (acc: 0.1333)
 Starting fold 7 / 10 (acc: 0.0667)
 Starting fold 8 / 10 (acc: 0.2000)
 Starting fold 9 / 10 (acc: 0.1333)
 Starting fold 10 / 10 (acc: 0.0667)
Mean accuracy on test set: 0.1600

Training with 250 estimators
 Starting fold 1 / 10 (acc: 0.1333)
 Starting fold 2 / 10 (acc: 0.1333)
 Starting fold 3 / 10 (acc: 0.2000)
 Starting fold 4 / 10 (acc: 0.2000)
 Starting fold 5 / 10 (acc: 0.2000)
 Starting fold 6 / 10 (acc: 0.2000)
 Starting fold 7 / 10 (acc: 0.2000)
 Starting fold 8 / 10 (acc: 0.2667)
 Starting fold 9 / 10 (acc: 0.2667)
 Starting fold 10 / 10 (acc: 0.2000)
Mean accuracy on test set: 0.2000

Training with 500 estimators
 Starting fold 1 / 10 (acc: 0.1333)
 Starting fold 2 / 10 

In [8]:
print("=== Results of supervised learning on crowd likert labels ===")
random_forest_acc_crowd_likert = random_forest(X_2D,
                                               y_likert_crowd,
                                               likert_crowd_idx)

=== Results of supervised learning on crowd likert labels ===
Training with 100 estimators
 Starting fold 1 / 10 (acc: 0.2500)
 Starting fold 2 / 10 (acc: 0.2500)
 Starting fold 3 / 10 (acc: 0.3125)
 Starting fold 4 / 10 (acc: 0.2500)
 Starting fold 5 / 10 (acc: 0.3750)
 Starting fold 6 / 10 (acc: 0.3125)
 Starting fold 7 / 10 (acc: 0.1875)
 Starting fold 8 / 10 (acc: 0.3125)
 Starting fold 9 / 10 (acc: 0.2500)
 Starting fold 10 / 10 (acc: 0.3125)
Mean accuracy on test set: 0.2812

Training with 250 estimators
 Starting fold 1 / 10 (acc: 0.1875)
 Starting fold 2 / 10 (acc: 0.2500)
 Starting fold 3 / 10 (acc: 0.4375)
 Starting fold 4 / 10 (acc: 0.2500)
 Starting fold 5 / 10 (acc: 0.3750)
 Starting fold 6 / 10 (acc: 0.2500)
 Starting fold 7 / 10 (acc: 0.2500)
 Starting fold 8 / 10 (acc: 0.2500)
 Starting fold 9 / 10 (acc: 0.1875)
 Starting fold 10 / 10 (acc: 0.2500)
Mean accuracy on test set: 0.2687

Training with 500 estimators
 Starting fold 1 / 10 (acc: 0.2500)
 Starting fold 2 / 10 (

In [9]:
print("=== Results of supervised learning on expert dominant labels ===")
random_forest_acc_experts_dominant = random_forest(X_2D,
                                                   y_dominant_experts, 
                                                   dominant_expert_idx)

=== Results of supervised learning on expert dominant labels ===
Training with 100 estimators
 Starting fold 1 / 10 (acc: 0.6667)
 Starting fold 2 / 10 (acc: 0.6667)
 Starting fold 3 / 10 (acc: 0.4444)
 Starting fold 4 / 10 (acc: 0.7778)
 Starting fold 5 / 10 (acc: 0.6667)
 Starting fold 6 / 10 (acc: 0.7778)
 Starting fold 7 / 10 (acc: 0.6667)
 Starting fold 8 / 10 (acc: 0.6667)
 Starting fold 9 / 10 (acc: 0.7778)
 Starting fold 10 / 10 (acc: 0.6667)
Mean accuracy on test set: 0.6778

Training with 250 estimators
 Starting fold 1 / 10 (acc: 0.6667)
 Starting fold 2 / 10 (acc: 0.6667)
 Starting fold 3 / 10 (acc: 0.7778)
 Starting fold 4 / 10 (acc: 0.6667)
 Starting fold 5 / 10 (acc: 0.7778)
 Starting fold 6 / 10 (acc: 0.6667)
 Starting fold 7 / 10 (acc: 0.6667)
 Starting fold 8 / 10 (acc: 0.7778)
 Starting fold 9 / 10 (acc: 0.6667)
 Starting fold 10 / 10 (acc: 0.6667)
Mean accuracy on test set: 0.7000

Training with 500 estimators
 Starting fold 1 / 10 (acc: 0.6667)
 Starting fold 2 / 1

In [10]:
print("=== Results of supervised learning on crowd dominant labels ===")
random_forest_acc_crowd_dominant = random_forest(X_2D,
                                                 y_dominant_crowd,
                                                 dominant_crowd_idx)

=== Results of supervised learning on crowd dominant labels ===
Training with 100 estimators
 Starting fold 1 / 10 (acc: 0.5714)
 Starting fold 2 / 10 (acc: 0.5714)
 Starting fold 3 / 10 (acc: 0.6429)
 Starting fold 4 / 10 (acc: 0.5714)
 Starting fold 5 / 10 (acc: 0.6429)
 Starting fold 6 / 10 (acc: 0.7143)
 Starting fold 7 / 10 (acc: 0.5714)
 Starting fold 8 / 10 (acc: 0.5000)
 Starting fold 9 / 10 (acc: 0.6429)
 Starting fold 10 / 10 (acc: 0.5000)
Mean accuracy on test set: 0.5929

Training with 250 estimators
 Starting fold 1 / 10 (acc: 0.7857)
 Starting fold 2 / 10 (acc: 0.5714)
 Starting fold 3 / 10 (acc: 0.5714)
 Starting fold 4 / 10 (acc: 0.7143)
 Starting fold 5 / 10 (acc: 0.5714)
 Starting fold 6 / 10 (acc: 0.6429)
 Starting fold 7 / 10 (acc: 0.6429)
 Starting fold 8 / 10 (acc: 0.7143)
 Starting fold 9 / 10 (acc: 0.5000)
 Starting fold 10 / 10 (acc: 0.5714)
Mean accuracy on test set: 0.6286

Training with 500 estimators
 Starting fold 1 / 10 (acc: 0.6429)
 Starting fold 2 / 10

# PyTorch Preparations

In [11]:
## convert numpy arrays to PyTorch tensors
X_2D = torch.from_numpy(X_2D)
X_3D = torch.from_numpy(X_3D)
y_likert_crowd = torch.from_numpy(y_likert_crowd)
y_likert_experts = torch.from_numpy(y_likert_experts)
y_dominant_crowd = torch.from_numpy(y_dominant_crowd)
y_dominant_experts = torch.from_numpy(y_dominant_experts)

In [12]:
def categorical_accuracy(y_hat, y):
    # y := 1D array of class labels
    # y_hat := 2D array of one-hot class labels
    _, labels = y_hat.max(dim=1)
    return torch.mean(torch.eq(labels, y).float())

def fit(model, X, y, index, lr=0.01, l2norm=0.001, n_folds=10, n_epoch=250, patience=7):
    n_samples = X[index].shape[0]

    loss = 0
    acc = 0
    for fold_i in range(n_folds):
        print("Starting fold {} / {}".format(fold_i+1, n_folds), end='')
        model.init()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)
        
        # early stopping
        patience_left = patience
        best_score = -1
        delta = 1e-4
        best_state = None
        
        train_fold_idx, test_fold_idx  = create_splits(y[index])
        train_idx = index[train_fold_idx]
        test_idx = index[test_fold_idx]
        for epoch in range(n_epoch):
            model.train()
            
            y_hat = model(X[train_idx].float())
            train_acc = categorical_accuracy(y_hat, y[train_idx])
            train_loss = criterion(y_hat, y[train_idx].long())
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()
            
            model.eval()
            test_loss = None
            with torch.no_grad():
                y_hat = model(X[test_idx].float())
                test_acc = categorical_accuracy(y_hat, y[test_idx])
                test_loss = criterion(y_hat, y[test_idx].long())
                
            train_loss = float(train_loss.item())
            test_loss = float(test_loss.item())
            
            if patience <= 0:
                continue
            if best_score < 0:
                best_score = test_loss
                best_state = model.state_dict()
            if test_loss >= best_score - delta:
                patience_left -= 1
            else:
                best_score = test_loss
                best_state = model.state_dict()
                patience_left = patience
            if patience_left <= 0:
                model.load_state_dict(best_state)
                
                test_idx = index[create_splits(y[index])[1]]  # get new random test set to validate on
                with torch.no_grad():
                    y_hat = model(X[test_idx].float())
                    test_acc = categorical_accuracy(y_hat, y[test_idx])
                    test_loss = float(criterion(y_hat, y[test_idx].long()).item())
        
        loss += test_loss
        acc += test_acc
        print(" - training accuracy: {:.4f} / loss: {:.4f} - test accuracy: {:.4f} / loss: {:.4f}".format(train_acc,
                                                                                          train_loss,
                                                                                          test_acc,
                                                                                          test_loss))
        
    loss /= n_folds
    acc /= n_folds
    print("average loss on test set: {:.4f}".format(loss))
    print("average accuracy on test set: {:.4f}".format(acc))
    
    return acc

# Neural Network

In [13]:
class ClassifierNN(nn.Module):
    """Simple Neural Network Classifier"""

    def __init__(self, input_dim, output_dim, p_dropout=0.05):
        super().__init__()
        hidden_dim = (input_dim-output_dim)//2
        
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p=p_dropout),
            

            nn.Linear(hidden_dim, output_dim),
            nn.ReLU(inplace=True))
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, X):
        return self.softmax(self.fc(X))
        
    def init(self):
        for param in self.parameters():
            nn.init.normal_(param)

In [14]:
## hyperparameters
lr = 0.01
p_dropout = 0.05

## define model
indim = X_2D.shape[1]
outdim = np.unique(y_likert_experts[likert_expert_idx]).shape[0]
assert outdim == np.unique(y_likert_crowd[likert_crowd_idx]).shape[0]

model = ClassifierNN(input_dim=indim,
                     output_dim=outdim,
                     p_dropout=p_dropout)

print("=== Results on expert likert labels ===")
neural_net_acc_likert_experts = fit(model, X_2D, y_likert_experts, likert_expert_idx, lr=lr)

print("\n=== Results on crowd likert labels ===")
neural_net_acc_likert_crowd = fit(model, X_2D, y_likert_crowd, likert_crowd_idx, lr=lr)

=== Results on expert likert labels ===
Starting fold 1 / 10 - training accuracy: 0.8372 / loss: 1.3278 - test accuracy: 0.2000 / loss: 1.9281
Starting fold 2 / 10 - training accuracy: 0.9535 / loss: 1.2066 - test accuracy: 0.0000 / loss: 2.1007
Starting fold 3 / 10 - training accuracy: 0.6977 / loss: 1.4595 - test accuracy: 0.0667 / loss: 2.0391
Starting fold 4 / 10 - training accuracy: 0.8605 / loss: 1.3033 - test accuracy: 0.0667 / loss: 2.0832
Starting fold 5 / 10 - training accuracy: 0.9302 / loss: 1.2385 - test accuracy: 0.1333 / loss: 1.9407
Starting fold 6 / 10 - training accuracy: 0.9070 / loss: 1.2575 - test accuracy: 0.0667 / loss: 2.1006
Starting fold 7 / 10 - training accuracy: 0.7442 / loss: 1.4175 - test accuracy: 0.2000 / loss: 1.8992
Starting fold 8 / 10 - training accuracy: 0.8837 / loss: 1.2793 - test accuracy: 0.0000 / loss: 2.0930
Starting fold 9 / 10 - training accuracy: 0.8837 / loss: 1.2792 - test accuracy: 0.0000 / loss: 2.1144
Starting fold 10 / 10 - training 

In [15]:
## hyperparameters
lr = 0.01
p_dropout = 0.05

## define model
indim = X_2D.shape[1]
outdim = np.unique(y_dominant_experts[dominant_expert_idx]).shape[0]
assert outdim == np.unique(y_dominant_crowd[dominant_crowd_idx]).shape[0]

model = ClassifierNN(input_dim=indim,
                     output_dim=outdim,
                     p_dropout=p_dropout)

print("=== Results on expert dominant labels ===")
neural_net_acc_dominant_experts = fit(model, X_2D, y_dominant_experts, dominant_expert_idx, lr=lr)

print("\n=== Results on crowd dominant labels ===")
neural_net_acc_dominant_crowd = fit(model, X_2D, y_dominant_crowd, dominant_crowd_idx, lr=lr)

=== Results on expert dominant labels ===
Starting fold 1 / 10 - training accuracy: 0.9714 / loss: 0.3517 - test accuracy: 0.6667 / loss: 0.6826
Starting fold 2 / 10 - training accuracy: 1.0000 / loss: 0.4340 - test accuracy: 0.4444 / loss: 0.7341
Starting fold 3 / 10 - training accuracy: 0.9429 / loss: 0.3704 - test accuracy: 0.5556 / loss: 0.7052
Starting fold 4 / 10 - training accuracy: 0.6857 / loss: 0.6275 - test accuracy: 0.6667 / loss: 0.6466
Starting fold 5 / 10 - training accuracy: 0.9714 / loss: 0.3458 - test accuracy: 0.6667 / loss: 0.6457
Starting fold 6 / 10 - training accuracy: 0.8857 / loss: 0.4703 - test accuracy: 0.6667 / loss: 0.6774
Starting fold 7 / 10 - training accuracy: 0.8571 / loss: 0.4559 - test accuracy: 0.7778 / loss: 0.5920
Starting fold 8 / 10 - training accuracy: 0.9714 / loss: 0.3428 - test accuracy: 0.6667 / loss: 0.6988
Starting fold 9 / 10 - training accuracy: 0.9429 / loss: 0.3618 - test accuracy: 0.4444 / loss: 0.8672
Starting fold 10 / 10 - trainin

# CNN

In [16]:
class ClassifierCNN(nn.Module):
    """CNN Classifier"""

    def __init__(self, features_in, features_out, p_dropout=0.05):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(features_in, int(features_in*1.5), kernel_size=7),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3), 

            nn.Conv1d(int(features_in*1.5), int(features_in*2), kernel_size=7),
            nn.ReLU(inplace=True),
            
            nn.Conv1d(int(features_in*2), int(features_in*2), kernel_size=5),
            nn.ReLU(inplace=True),
            nn.AdaptiveMaxPool1d(2)
        )

        self.fc = nn.Sequential(
            nn.Linear(int(features_in*2)*2, 32),
            nn.ReLU(inplace=True),
            nn.Dropout(p=p_dropout),

            nn.Linear(32, features_out)
        )
        
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, X):
        X = self.conv(X)
        X = X.view(X.size(0), -1)

        return self.softmax(self.fc(X))
        
    def init(self):
        for param in self.parameters():
            nn.init.normal_(param)

In [17]:
## hyperparameters
lr = 0.01
p_dropout = 0.05

## define model
indim = X_3D.shape[2]
outdim = np.unique(y_likert_experts[likert_expert_idx]).shape[0]
assert outdim == np.unique(y_likert_crowd[likert_crowd_idx]).shape[0]

model = ClassifierCNN(features_in=indim,
                      features_out=outdim,
                      p_dropout=p_dropout)

print("=== Results on expert likert labels ===")
cnn_acc_likert_experts = fit(model, X_3D.transpose(1, 2), y_likert_experts, likert_expert_idx, lr=lr)

print("\n=== Results on crowd likert labels ===")
cnn_acc_likert_crowd = fit(model, X_3D.transpose(1, 2), y_likert_crowd, likert_crowd_idx, lr=lr)

=== Results on expert likert labels ===
Starting fold 1 / 10 - training accuracy: 0.2558 / loss: 1.9096 - test accuracy: 0.2000 / loss: 1.9654
Starting fold 2 / 10 - training accuracy: 0.0930 / loss: 2.0724 - test accuracy: 0.1333 / loss: 2.0321
Starting fold 3 / 10 - training accuracy: 0.1163 / loss: 2.0491 - test accuracy: 0.1333 / loss: 2.0321
Starting fold 4 / 10 - training accuracy: 0.0930 / loss: 2.0724 - test accuracy: 0.1333 / loss: 2.0321
Starting fold 5 / 10 - training accuracy: 0.2093 / loss: 1.9561 - test accuracy: 0.2000 / loss: 1.9654
Starting fold 6 / 10 - training accuracy: 0.1395 / loss: 2.0259 - test accuracy: 0.1333 / loss: 2.0321
Starting fold 7 / 10 - training accuracy: 0.1628 / loss: 2.0026 - test accuracy: 0.1333 / loss: 2.0321
Starting fold 8 / 10 - training accuracy: 0.0000 / loss: 2.1654 - test accuracy: 0.0667 / loss: 2.0988
Starting fold 9 / 10 - training accuracy: 0.1395 / loss: 2.0259 - test accuracy: 0.1333 / loss: 2.0321
Starting fold 10 / 10 - training 

In [18]:
## hyperparameters
lr = 0.01
p_dropout = 0.05

## define model
indim = X_3D.shape[2]
outdim = np.unique(y_likert_experts[likert_expert_idx]).shape[0]
assert outdim == np.unique(y_likert_crowd[likert_crowd_idx]).shape[0]

model = ClassifierCNN(features_in=indim,
                      features_out=outdim,
                      p_dropout=p_dropout)

print("=== Results on expert dominant labels ===")
cnn_acc_dominant_experts = fit(model, X_3D.transpose(1, 2), y_dominant_experts, dominant_expert_idx, lr=lr)

print("\n=== Results on crowd dominant labels ===")
cnn_acc_dominant_crowd = fit(model, X_3D.transpose(1, 2), y_dominant_crowd, dominant_crowd_idx, lr=lr)

=== Results on expert dominant labels ===
Starting fold 1 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 2 / 10 - training accuracy: 0.0000 / loss: 2.1654 - test accuracy: 0.0000 / loss: 2.1654
Starting fold 3 / 10 - training accuracy: 0.0000 / loss: 2.1654 - test accuracy: 0.0000 / loss: 2.1654
Starting fold 4 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 5 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 6 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 7 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 8 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 9 / 10 - training accuracy: 0.0000 / loss: 2.1654 - test accuracy: 0.0000 / loss: 2.1654
Starting fold 10 / 10 - trainin

# LSTM

In [19]:
class ClassifierLSTM(nn.Module):
    def __init__(self,
                 input_dim,
                 output_dim,
                 hidden_dim,
                 num_layers=1,
                 p_dropout=0.0):
        """
        LSTM

        """
        super().__init__()
        self.hidden_dim = hidden_dim

        self.lstm = nn.LSTM(input_size=input_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            bias=True,
                            batch_first=True)  # (batch, seq, feature)
                            
        fc_hidden_dim = (hidden_dim-output_dim)//2
        self.fc = nn.Sequential(nn.Linear(hidden_dim, fc_hidden_dim),
                                nn.ReLU(inplace=True),
                                nn.Dropout(p=p_dropout),
                                
                                nn.Linear(fc_hidden_dim, output_dim))
        
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        # default H0 is zero vector
        # output Hn is representation of entire sequence
        X, _ = self.lstm(X)
        X = X[:,-1,:]  # only consider final output

        return self.softmax(self.fc(X))

    def init(self):
        sqrt_k = sqrt(1.0/self.hidden_dim)
        for param in self.parameters():
            nn.init.uniform_(param, -sqrt_k, sqrt_k)


In [20]:
## hyperparameters
lr = 0.01
p_dropout = 0.05

## define model
indim = X_3D.shape[2]
outdim = np.unique(y_likert_experts[likert_expert_idx]).shape[0]
assert outdim == np.unique(y_likert_crowd[likert_crowd_idx]).shape[0]
hidden_dim = (indim-outdim)//2

model = ClassifierLSTM(input_dim=indim,
                       output_dim=outdim,
                       hidden_dim=hidden_dim,
                       p_dropout=p_dropout)

print("=== Results on expert likert labels ===")
lstm_acc_likert_experts = fit(model, X_3D, y_likert_experts, likert_expert_idx, lr=lr)

print("\n=== Results on crowd likert labels ===")
lstm_acc_likert_crowd = fit(model, X_3D, y_likert_crowd, likert_crowd_idx, lr=lr)

=== Results on expert likert labels ===
Starting fold 1 / 10 - training accuracy: 0.2791 / loss: 1.8700 - test accuracy: 0.2000 / loss: 1.9163
Starting fold 2 / 10 - training accuracy: 0.2791 / loss: 1.8688 - test accuracy: 0.2000 / loss: 1.9160
Starting fold 3 / 10 - training accuracy: 0.2558 / loss: 1.8711 - test accuracy: 0.2000 / loss: 1.9177
Starting fold 4 / 10 - training accuracy: 0.2558 / loss: 1.8734 - test accuracy: 0.2000 / loss: 1.9168
Starting fold 5 / 10 - training accuracy: 0.3023 / loss: 1.8692 - test accuracy: 0.2000 / loss: 1.9161
Starting fold 6 / 10 - training accuracy: 0.1860 / loss: 1.8842 - test accuracy: 0.2000 / loss: 1.9160
Starting fold 7 / 10 - training accuracy: 0.3023 / loss: 1.8693 - test accuracy: 0.2000 / loss: 1.9161
Starting fold 8 / 10 - training accuracy: 0.2558 / loss: 1.8737 - test accuracy: 0.2000 / loss: 1.9167
Starting fold 9 / 10 - training accuracy: 0.2558 / loss: 1.8719 - test accuracy: 0.2000 / loss: 1.9160
Starting fold 10 / 10 - training 

In [21]:
## hyperparameters
lr = 0.01
p_dropout = 0.05

## define model
indim = X_3D.shape[2]
outdim = np.unique(y_likert_experts[likert_expert_idx]).shape[0]
assert outdim == np.unique(y_likert_crowd[likert_crowd_idx]).shape[0]

model = ClassifierLSTM(input_dim=indim,
                       output_dim=outdim,
                       hidden_dim=indim,
                       p_dropout=p_dropout)

print("=== Results on expert dominant labels ===")
lstm_acc_dominant_experts = fit(model, X_3D, y_dominant_experts, dominant_expert_idx, lr=lr)

print("\n=== Results on crowd dominant labels ===")
lstm_acc_dominant_crowd = fit(model, X_3D, y_dominant_crowd, dominant_crowd_idx, lr=lr)

=== Results on expert dominant labels ===
Starting fold 1 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 2 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 3 / 10 - training accuracy: 0.6857 / loss: 1.4828 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 4 / 10 - training accuracy: 0.6857 / loss: 1.4802 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 5 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 6 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 7 / 10 - training accuracy: 0.6857 / loss: 1.4807 - test accuracy: 0.6667 / loss: 1.4988
Starting fold 8 / 10 - training accuracy: 0.6857 / loss: 1.4807 - test accuracy: 0.6667 / loss: 1.4990
Starting fold 9 / 10 - training accuracy: 0.6857 / loss: 1.4797 - test accuracy: 0.6667 / loss: 1.4989
Starting fold 10 / 10 - trainin

# Transformer